In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("btc_price_prediction") \
    .config(
        "spark.jars.packages",
        "org.postgresql:postgresql:42.7.3"
    ) \
    .getOrCreate()




Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1139)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1125)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:489)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1795)
	at org.apache.spark.SparkContext.$anonfun$new$16(SparkContext.scala:533)
	at org.apache.spark.SparkContext.$anonfun$new$16$adapted(SparkContext.scala:533)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:533)
	at org.apache.spark.api.java.JavaSparkC

In [ ]:
#spark.stop()

## 1. Data Injection

In [ ]:
import requests
import os
from dotenv import load_dotenv
load_dotenv() 
def data_collection():

  SYMBOL = 'BTCUSDT'
  INTERVAL = '1m'   # Intervalle d'une minute
  LIMIT = 600    
  api='https://api.binance.com/api/v3/klines'
  response = requests.get(api, params={
    "symbol": SYMBOL,
    "interval": INTERVAL,
    "limit": LIMIT
  })
  if response.status_code != 200:
        print(f"Erreur {response.status_code}")
        return None
  api_data=response.json()
  columns = [
        "open_time", "open_price", "high_price", "low_price", "close_price", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
    ]
  psdf= spark.createDataFrame(api_data, columns)
 
  
  return psdf
  
data = data_collection()
data.show()

In [ ]:
data.printSchema()

In [ ]:
""""
from pyspark.sql.functions import col
import pyspark.sql.functions as sf
num_cols= ["open_price", "high_price", "low_price", "close_price", "volume", "quote_asset_volume", "number_of_trades", "taker_buy_base_volume", "taker_buy_quote_volume"]
data = data.select([col(feature).cast('double') for feature in num_cols])
data= data.select([col("open_time"), sf.timestamp_millis("open_time")])
data.printSchema()
"""




In [ ]:
from pyspark.sql.functions import col
num_cols= ["open_price", "high_price", "low_price", "close_price", "volume", "quote_asset_volume", "number_of_trades", "taker_buy_base_volume", "taker_buy_quote_volume"]
for feature in num_cols:
      data = data.withColumn(feature, col(feature).cast('double'))

data.printSchema()

In [ ]:
from pyspark.sql.functions import from_unixtime , to_timestamp

data = data.withColumn("open_time_ts", to_timestamp(from_unixtime(col("open_time")/1000))) \
       .withColumn("close_time_ts", to_timestamp(from_unixtime(col("close_time")/1000)))
data.printSchema()
data.show(5)
